In [1]:
#!pip install spotipy

In [2]:
# Acknowledgements:
# https://medium.com/swlh/how-to-leverage-spotify-api-genius-lyrics-for-data-science-tasks-in-python-c36cdfb55cf3

In [3]:
import pandas as pd
import re
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup as BS
import requests
import time
from unidecode import unidecode #Will make it easier to deal with songs with non english alphabet characters

In [4]:
cid = 'b1e13d47b67249bda69ed912bb4f7133'
secret = 'a733eca54ee84bb1b8d95b67cda8c91e'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [5]:
artist = input() #Type name of the artist to search for "This is {artist}" playlist

playlist = f"This is {artist}"
results = sp.search(q='playlist:' + playlist, type='playlist') #Results of the query
uri_ti_artist = str(results['playlists']['items'][0]['uri']) # URI of "This is {artist}" playlist
uri_ti_artist

Judas Priest


'spotify:playlist:37i9dQZF1DZ06evO1q6zh6'

In [6]:
# I'll get the track IDs from the "This is {artist}" playlist

def getTrackIDs(playlist_id):
    ids = []
    playlist = sp.playlist(playlist_id)
    for song in playlist['tracks']['items']:
        track = song['track']
        ids.append(track['id'])
    print('I could find {num_tracks} songs'.format(num_tracks=len(ids)))
    return ids

ids = getTrackIDs(uri_ti_artist)

I could find 50 songs


In [7]:
# Next, I build a function to retrieve all the useful info for my dataset:
unwanted = "[',!@#$;:!*%)(&^~]"

def getTrackFeatures(item):
    metadata = sp.track(item)
    features = sp.audio_features(item)
    
    #metadata:
    name = unidecode(metadata['name'])
    name = re.sub(r'[\(\[].*?[\)\]]', '', name) #Delete regular expressions from name
    name = re.sub(unwanted,'', name) #Delete other undesired regular expressions
    
    album = unidecode(metadata['album']['name'])
    album_cover = metadata['album']['images'][0]['url']
    artist = unidecode(metadata['album']['artists'][0]['name'])
    release_date = metadata['album']['release_date']
    length = metadata['duration_ms']
    popularity = metadata['popularity']
    sample = metadata['preview_url']
    
    #audio analysis features:
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']
    
    track = [name,album,album_cover,artist,sample,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature]
    return track

In [8]:
# Now I loop over track ids:
tracks = []
for item in ids:
    track = getTrackFeatures(item)
    #time.sleep(3)
    tracks.append(track)

In [9]:
tracks

[['Breaking the Law',
  'British Steel',
  'https://i.scdn.co/image/ab67616d0000b273be0e5ebab8c61469b1ff9f62',
  'Judas Priest',
  'https://p.scdn.co/mp3-preview/cb5f2bf729cf7b2e1517d81258ec74c76c4aadb9?cid=b1e13d47b67249bda69ed912bb4f7133',
  '1980',
  153840,
  73,
  0.024,
  0.362,
  0.961,
  9.35e-06,
  0.159,
  -5.664,
  0.0857,
  164.019,
  4],
 ['Youve Got Another Thing Coming',
  'Screaming For Vengeance (Expanded Edition)',
  'https://i.scdn.co/image/ab67616d0000b273475bd4fb40ba687fbf002c82',
  'Judas Priest',
  'https://p.scdn.co/mp3-preview/a865bf18cbc2c8787ef1544d4154103b773276d6?cid=b1e13d47b67249bda69ed912bb4f7133',
  '1982',
  310080,
  67,
  0.00572,
  0.519,
  0.833,
  4.33e-06,
  0.104,
  -6.947,
  0.0496,
  138.225,
  4],
 ['Painkiller',
  'Painkiller',
  'https://i.scdn.co/image/ab67616d0000b27360db4ca924d17bc6754e89aa',
  'Judas Priest',
  'https://p.scdn.co/mp3-preview/8300d443f522eecdbfbc3a589c34a8d731e2f3a2?cid=b1e13d47b67249bda69ed912bb4f7133',
  '1990-09-26',


In [10]:
# Now, I add lyrics scraping from genius.com
def scrape_lyrics(artist,song):
    artistname = str(artist.replace(' ','-') if ' ' in artist else str(artist))
    
    #Cleaning of non useful info
    song = str(song.replace(" - 2015 Remaster","") if " - 2015 Remaster" in song else str(song))
    song = str(song.replace(" - Remastered","") if " - Remastered" in song else str(song))
    songname = str(song.replace(' ','-') if ' ' in song else str(song))
    
    page = requests.get('https://genius.com/'+ artistname + '-' + songname + '-' + 'lyrics',headers={'User-Agent': 'Mozilla/5.0'})
    print('https://genius.com/'+ artistname + '-' + songname + '-' + 'lyrics')
    html = BS(page.text,'html.parser')
    #print(html)
    #time.sleep(3)
    lyrics = html.find_all('div', {"data-lyrics-container":"true","class":'Lyrics__Container-sc-1ynbvzw-6 YYrds'})
    #print(lyrics)
    lyrics_list = []
    for text in lyrics:
        #remove identifiers like chorus, verse, etc
        verse = text.get_text(separator=" ").strip()
        verse = re.sub(r'[\(\[].*?[\)\]]', '', verse)
        verse = re.sub(unwanted, '', verse)
        #print(verse)
        lyrics_list.append(verse)
    lyrics_text = "\n".join(lyrics_list)
    return lyrics_text

In [11]:
# create dataset
df = pd.DataFrame(tracks, columns = ["track","album","album_cover","artist","sample","release_date","length","popularity","acousticness","danceability","energy","instrumentalness","liveness","loudness","speechiness","tempo","time_signature"])
df

track  \
0                 Breaking the Law   
1   Youve Got Another Thing Coming   
2                       Painkiller   
3                     No Surrender   
4            Living After Midnight   
5                      The Hellion   
6       Heading Out to the Highway   
7         Turbo Lover - Remastered   
8                        Firepower   
9                     Electric Eye   
10                     Hell Patrol   
11                           Angel   
12                Lightning Strike   
13      Beyond the Realms of Death   
14                   Night Crawler   
15                    The Sentinel   
16                 Before the Dawn   
17                      Metal Gods   
18                 A Touch of Evil   
19               Diamonds and Rust   
20           Hell Bent for Leather   
21               Halls of Valhalla   
22                All Guns Blazing   
23                          United   
24                Never The Heroes   
25              Riding on the Wind   
26   Between the Hammer  the Anvil   
27                         Grinder   
28                      Jawbreaker   
29            The Green Manalishi    
30           Prisoner of Your Eyes   
31                   Desert Plains   
32      The Hellion / Electric Eye   
33                      Rapid Fire   
34         Screaming for Vengeance   
35                 Johnny B. Goode   
36             Rock Hard Ride Free   
37                      Hot Rockin   
38                       Guardians   
39    Better by You Better Than Me   
40                      Love Bites   
41                      Bloodstone   
42             Children of the Sun   
43                     Ram It Down   
44               Freewheel Burning   
45                          Chains   
46                    Judas Rising   
47                 Blood Red Skies   
48       Some Heads Are Gonna Roll   
49             Dissident Aggressor   

                                         album  \
0                                British Steel   
1   Screaming For Vengeance (Expanded Edition)   
2                                   Painkiller   
3                                    Firepower   
4                                British Steel   
5   Screaming For Vengeance (Expanded Edition)   
6                               Point Of Entry   
7                                     Turbo 30   
8                                    Firepower   
9   Screaming For Vengeance (Expanded Edition)   
10                                  Painkiller   
11                        Angel Of Retribution   
12                                   Firepower   
13                               Stained Class   
14                                  Painkiller   
15                      Defenders Of The Faith   
16                             Killing Machine   
17                               British Steel   
18                                  Painkiller   
19                               Sin After Sin   
20                             Killing Machine   
21                  Redeemer of Souls (Deluxe)   
22                                  Painkiller   
23                               British Steel   
24                                   Firepower   
25                     Screaming For Vengeance   
26                                  Painkiller   
27                               British Steel   
28                      Defenders Of The Faith   
29                             Killing Machine   
30                     Screaming For Vengeance   
31                              Point Of Entry   
32                       Living After Midnight   
33                               British Steel   
34  Screaming For Vengeance (Expanded Edition)   
35                                 Ram It Down   
36                      Defenders Of The Faith   
37                              Point Of Entry   
38                                   Firepower   
39                               Stained Class   
40                      Defenders Of The

In [12]:
test = scrape_lyrics(artist,df['track'].values[5])
test

https://genius.com/Judas-Priest-The-Hellion-lyrics


''

In [13]:
def lyrics_onto_dataframe(df,artist):
    for i,song in enumerate(df['track']):
        test = scrape_lyrics(artist,song)
        #print(test)
        df.loc[i,'lyrics'] = test
    return df

df = lyrics_onto_dataframe(df,artist)

https://genius.com/Judas-Priest-Breaking-the-Law-lyrics
https://genius.com/Judas-Priest-Youve-Got-Another-Thing-Coming-lyrics
https://genius.com/Judas-Priest-Painkiller-lyrics
https://genius.com/Judas-Priest-No-Surrender-lyrics
https://genius.com/Judas-Priest-Living-After-Midnight-lyrics
https://genius.com/Judas-Priest-The-Hellion-lyrics
https://genius.com/Judas-Priest-Heading-Out-to-the-Highway-lyrics
https://genius.com/Judas-Priest-Turbo-Lover-lyrics
https://genius.com/Judas-Priest-Firepower-lyrics
https://genius.com/Judas-Priest-Electric-Eye-lyrics
https://genius.com/Judas-Priest-Hell-Patrol-lyrics
https://genius.com/Judas-Priest-Angel-lyrics
https://genius.com/Judas-Priest-Lightning-Strike-lyrics
https://genius.com/Judas-Priest-Beyond-the-Realms-of-Death-lyrics
https://genius.com/Judas-Priest-Night-Crawler-lyrics
https://genius.com/Judas-Priest-The-Sentinel-lyrics
https://genius.com/Judas-Priest-Before-the-Dawn-lyrics
https://genius.com/Judas-Priest-Metal-Gods-lyrics
https://genius

In [14]:
path=f"C:\\Users\\mdds2\\OneDrive\\Escritorio\\Proyectos-en-data-science\\Music and lyrics recommendation system and generator (under construction)\\generated_data\\{artist}.csv"
df.to_csv(path, sep = ',')
tracks_data = pd.read_csv(path)
tracks_data

Unnamed: 0                           track  \
0            0                Breaking the Law   
1            1  Youve Got Another Thing Coming   
2            2                      Painkiller   
3            3                    No Surrender   
4            4           Living After Midnight   
5            5                     The Hellion   
6            6      Heading Out to the Highway   
7            7        Turbo Lover - Remastered   
8            8                       Firepower   
9            9                    Electric Eye   
10          10                     Hell Patrol   
11          11                           Angel   
12          12                Lightning Strike   
13          13      Beyond the Realms of Death   
14          14                   Night Crawler   
15          15                    The Sentinel   
16          16                 Before the Dawn   
17          17                      Metal Gods   
18          18                 A Touch of Evil   
19          19               Diamonds and Rust   
20          20           Hell Bent for Leather   
21          21               Halls of Valhalla   
22          22                All Guns Blazing   
23          23                          United   
24          24                Never The Heroes   
25          25              Riding on the Wind   
26          26   Between the Hammer  the Anvil   
27          27                         Grinder   
28          28                      Jawbreaker   
29          29            The Green Manalishi    
30          30           Prisoner of Your Eyes   
31          31                   Desert Plains   
32          32      The Hellion / Electric Eye   
33          33                      Rapid Fire   
34          34         Screaming for Vengeance   
35          35                 Johnny B. Goode   
36          36             Rock Hard Ride Free   
37          37                      Hot Rockin   
38          38                       Guardians   
39          39    Better by You Better Than Me   
40          40                      Love Bites   
41          41                      Bloodstone   
42          42             Children of the Sun   
43          43                     Ram It Down   
44          44               Freewheel Burning   
45          45                          Chains   
46          46                    Judas Rising   
47          47                 Blood Red Skies   
48          48       Some Heads Are Gonna Roll   
49          49             Dissident Aggressor   

                                         album  \
0                                British Steel   
1   Screaming For Vengeance (Expanded Edition)   
2                                   Painkiller   
3                                    Firepower   
4                                British Steel   
5   Screaming For Vengeance (Expanded Edition)   
6                               Point Of Entry   
7                                     Turbo 30   
8                                    Firepower   
9   Screaming For Vengeance (Expanded Edition)   
10                                  Painkiller   
11                        Angel Of Retribution   
12                                   Firepower   
13                               Stained Class   
14                                  Painkiller   
15                      Defenders Of The Faith   
16                             Killing Machine   
17                               British Steel   
18                                  Painkiller   
19                               Sin After Sin   
20                             Killing Machine   
21                  Redeemer of Souls (Deluxe)   
22                                  Painkiller   
23                               British Steel   
24                                   Firepower   
25                     Screaming For Vengeance   
26                                  Painkiller   
27                               British Steel   
28 

In [15]:
class Track(object):
    def __init__(self,tracks_data):
        self.name = tracks_data['track']
        self.album = tracks_data['album']
        self.artist = tracks_data['artist']
        self.sample = tracks_data['sample']
        self.tempo = tracks_data['tempo']
        self.lyrics = tracks_data['lyrics']

In [16]:
tracks_data = tracks_data.apply(Track,axis=1)

In [17]:
tracks_data[9].lyrics

' Up here in space I’m looking down on you My lasers trace everything you do You think you’ve private lives  think nothing of the kind There is no true escape I’m watching all the time  I’m made of metal My circuits gleam I am perpetual I keep the country clean  I’m elected electric spy I’m protected electric eye  Always in focus  you can’t feel my stare I zoom into you but you don’t know I’m there I take a pride in probing all your secret moves My tearless retina takes pictures that can prove  I’m made of metal My circuits gleam I am perpetual I keep the country clean  I’m elected electric spy I’m protected electric eye   Electric eye in the sky Feel my stare always there There’s nothing you can do about it Develop and expose I feed upon your every thought And so my power grows\n I’m made of metal My circuits gleam I am perpetual I keep the country clean  I’m elected electric spy I’m protected electric eye  Im elected electric spy Im elected protected detective electric eye'